# 🏦 Project: Loan Eligibility Prediction
## 🚀 Phase 2: Data Preprocessing & Machine Learning Pipeline

---

### 📖 **Overview**
Welcome to the engine room of the project. After exploring the data in **Phase 1 (EDA)**, we now transition to building predictive models. The goal is to automate the loan eligibility process (real-time) based on customer details provided while filling out online application forms.

### 🎯 **The Mission**
To build a robust binary classifier that predicts `Loan_Status` (Approved/Rejected).
* **Business Goal:** Minimize risk for the bank while ensuring eligible applicants aren't turned away.
* **Key Metrics:** We prioritize **Accuracy** and **Weighted F1-Score** to balance precision and recall.

### ⚙️ **Notebook Workflow**
1.  **Preprocessing & Imputation:** Using `KNNImputer` for numerical gaps and Mode for categorical gaps.
2.  **Feature Engineering:** Log-transforming skewed financial data (`Total_Income`) and One-Hot Encoding categories.
3.  **Baseline Screening:** Testing **14 different algorithms** (Linear, Trees, Ensembles, SVMs) to find top performers.
4.  **Hyperparameter Tuning:** Using `GridSearchCV` to optimize the best candidates.
5.  **Final Selection:** Choosing the "Champion Model" for the final evaluation phase.

---

In [5]:
# --- Data Manipulation ---
import numpy as np
import pandas as pd

# --- Scikit-Learn: Preprocessing & Imputation ---
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer  # For Log scaling
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

# --- Scikit-Learn: Models ---
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# --- Scikit-Learn: Metrics ---
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, confusion_matrix, classification_report, precision_score, recall_score

# --- Configuration ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 2. 📂 Data Loading & Initial Inspection
We load the preprocessed dataset saved from the previous EDA phase.
* **Source:** `preprocessed_loan.csv`
* **Action:** verification of the first few rows to ensure data integrity.

In [6]:
# Load the dataset
df = pd.read_csv(r"..\data\preprocessed_loan.csv")

# Display the first 5 rows to check structure
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Total_Income,Loan_Amount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,6091.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,4941.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.0,141.0,360.0,1.0,Urban,Y


## 3. 🛠️ Feature Formatting & Target Encoding
Before splitting the data, we must ensure features have the correct data types.

1.  **Loan_Amount_Term:** Converted to `object` (categorical) because loan terms are discrete categories (e.g., 360 months, 180 months), not continuous numbers.
2.  **Credit_History:** Converted to `category` as it represents a binary state (0 or 1).
3.  **Loan_Status (Target):** We map the target variable `'Y'`/`'N'` to binary `1`/`0` for machine learning compatibility.

In [7]:
# 1. Cast Loan_Amount_Term to Int64 (handles NaNs) then to object (categorical)
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].astype('Int64')
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].astype('object')

# 2. Cast Credit_History to category
df['Credit_History'] = df['Credit_History'].astype('category')

# 3. Check distribution of the target variable before encoding
print("Target Class Distribution (Before Encoding):")
print(df['Loan_Status'].value_counts())

# 4. Encode Target: Y -> 1 (Approved), N -> 0 (Rejected)
df['Loan_Status'] = df['Loan_Status'].map({'Y':1, 'N':0})

# Verify the changes
df.info()

Target Class Distribution (Before Encoding):
Loan_Status
Y    422
N    192
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Loan_ID           614 non-null    object  
 1   Gender            601 non-null    object  
 2   Married           611 non-null    object  
 3   Dependents        599 non-null    object  
 4   Education         614 non-null    object  
 5   Self_Employed     582 non-null    object  
 6   Total_Income      614 non-null    float64 
 7   Loan_Amount       592 non-null    float64 
 8   Loan_Amount_Term  600 non-null    object  
 9   Credit_History    564 non-null    category
 10  Property_Area     614 non-null    object  
 11  Loan_Status       614 non-null    int64   
dtypes: category(1), float64(2), int64(1), object(8)
memory usage: 53.6+ KB


## 4. 📊 Feature Engineering Strategy
We separate our features into **Numerical** and **Categorical** groups. This is crucial because they require different preprocessing pipelines:
* **Numeric:** Requires scaling (to handle outliers like high incomes).
* **Categorical:** Requires encoding (to convert text labels to numbers).

### **Feature Groups:**
* **Target:** `Loan_Status`
* **Numeric:** `Total_Income`, `Loan_Amount`
* **Categorical:** Gender, Married, Dependents, Education, Self_Employed, Loan_Amount_Term, Credit_History, Property_Area.

In [8]:
# Define feature groups
target_feature = 'Loan_Status'

numeric_features = ['Total_Income', 'Loan_Amount']

categorical_features = ['Gender',
                        'Married',
                        'Dependents',
                        'Education',
                        'Self_Employed',
                        'Loan_Amount_Term',
                        'Credit_History',
                        'Property_Area'
                            ]

print(f"✅ Numeric Features: {numeric_features}")
print(f"✅ Categorical Features: {categorical_features}")

# Ensure all categorical features are strictly cast to type 'category'
# This saves memory and ensures compatibility with certain sklearn selectors
for col in categorical_features:
    df[col] = df[col].astype('category')

✅ Numeric Features: ['Total_Income', 'Loan_Amount']
✅ Categorical Features: ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']


## 5. ✂️ Data Splitting (Train-Validation-Test)
To build a robust model and prevent overfitting, we use a **three-way split strategy**:

1.  **Training Set (64%):** Used to fit the models.
2.  **Validation Set (16%):** Used for unbiased model evaluation and hyperparameter tuning during the development phase.
3.  **Test Set (20%):** Held out completely until the very end to provide a final performance estimate.

**Method:** We use `stratify=y` to ensure the proportion of Approved/Rejected loans remains consistent across all three datasets.

In [9]:
# Separate features (X) and target (y)
X = df.drop("Loan_Status", axis=1)
y = df["Loan_Status"]

# 1. First Split: Separate out the Test set (20%)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Essential for imbalanced datasets
)

# 2. Second Split: Separate the remaining 80% into Train and Validation
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_temp, y_train_temp,
    test_size=0.2, # 0.2 * 0.8 = 0.16 (16% of total data)
    random_state=42,
    stratify=y_train_temp
)

# Save sets to disk for reproducibility in other notebooks
X_test.to_csv(r'../data/X_test.csv', index=False)
y_test.to_csv(r'../data/y_test.csv', index=False)
X_train.to_csv(r"../data/X_train.csv", index=False)
y_train.to_csv(r"../data/y_train.csv", index=False)
X_valid.to_csv(r"../data/X_valid.csv", index=False)
y_valid.to_csv(r"../data/y_valid.csv", index=False)

# Check the shape of the resulting splits
print(f"Training Shape:   {X_train.shape}")
print(f"Validation Shape: {X_valid.shape}")
print(f"Test Shape:       {X_test.shape}")

Training Shape:   (392, 11)
Validation Shape: (99, 11)
Test Shape:       (123, 11)


## 6. ⚙️ Preprocessing Pipelines
Machine learning models cannot handle missing values or raw text. We build **Pipelines** to automate the cleanup process. This ensures that the exact same transformations applied to the training set are applied to the test set, preventing **data leakage**.

### **Transformation Strategy:**
1.  **Numeric Pipeline (`num`):**
    * **Imputation:** We use `KNNImputer` (K-Nearest Neighbors). Instead of just filling with the "average," this looks at similar borrowers to guess the missing income or loan amount.
    * **Scaling:** We use `np.log1p` (Log Transformation). Financial data (like Income) is often skewed. Log transformation makes it more "normal" (bell-curve shaped), which helps models like Logistic Regression and SVM.
2.  **Categorical Pipeline (`cat`):**
    * **Imputation:** We use `SimpleImputer(strategy='most_frequent')` to fill missing text with the most common category (Mode).
    * **Encoding:** We use `OneHotEncoder` to convert categories (e.g., "Graduate", "Not Graduate") into binary columns (1s and 0s).

In [10]:
# 1. Define distinct steps for numeric features
numeric_transformer = Pipeline(steps=[
    ('logtransformer', FunctionTransformer(np.log1p, validate=False)),  # Log transform for skewness
    ('imputer', KNNImputer(n_neighbors=5))                   # Fill missing using neighbors
])

# 2. Define distinct steps for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),     # Fill missing with mode
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # Convert to binary
])

# 3. Combine them into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ])

# Visualizing the Pipeline object
print("✅ Preprocessing pipeline created successfully.")

✅ Preprocessing pipeline created successfully.


## 7. 🧪 Baseline Model Screening
We don't know yet which algorithm will best understand the patterns in loan approvals. Therefore, we define a "dictionary" of distinct classifiers to test them all at once.

**We are testing 14 different algorithms across 4 families:**
1.  **Linear Models:** Logistic Regression, Ridge, SGD (Good baselines).
2.  **Tree-Based:** Decision Tree, Random Forest, Extra Trees (Good for capturing non-linear complex rules).
3.  **Boosting:** AdaBoost, Gradient Boosting (High performance, builds weak learners into strong ones).
4.  **Others:** SVM, KNN, Naive Bayes (Gaussian/Bernoulli), Discriminant Analysis.

In [11]:
# Dictionary of models to evaluate (with class_weight='balanced' where supported)
models = {
    # Linear & Distance based
    "Logistic Regression": LogisticRegression(class_weight='balanced'),
    "Ridge Classifier": RidgeClassifier(),        # does NOT support class_weight
    "SGD Classifier": SGDClassifier(class_weight='balanced'),
    "KNN": KNeighborsClassifier(),                # does NOT support class_weight
    "SVM": SVC(class_weight='balanced'),

    # Tree & Ensemble based
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced'),
    "Random Forest": RandomForestClassifier(class_weight='balanced'),
    "Extra Trees": ExtraTreesClassifier(),        # does NOT support class_weight
    "AdaBoost": AdaBoostClassifier(),             # does NOT support class_weight
    "Gradient Boosting": GradientBoostingClassifier(),  # does NOT support class_weight

    # Bayesian & Discriminant
    "GaussianNB": GaussianNB(),                   # does NOT support class_weight
    "BernoulliNB": BernoulliNB(),                 # does NOT support class_weight
    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(),   # no class_weight
    "Quadratic Discriminant Analysis": QuadraticDiscriminantAnalysis()  # no class_weight
}

print(f"✅ Initialized {len(models)} models (including class_weight='balanced' where supported).")

✅ Initialized 14 models (including class_weight='balanced' where supported).


## 8. 🏃‍♂️ Model Training & Evaluation Loop
Here, we iterate through our dictionary of 14 models. For each algorithm, we create a temporary **Pipeline** that:
1.  **Accepts raw data.**
2.  **Runs the Preprocessor** (imputes missing values, scales numbers, one-hot encodes text).
3.  **Fits the Model** on the `X_train` data.
4.  **Predicts** results on the `X_valid` (Validation) data.

**Metrics Used:**
* **Accuracy:** Overall correctness (Correct Predictions / Total Predictions).
* **F1 Score (Weighted):** The harmonic mean of Precision and Recall. This is often a better metric than accuracy for loan datasets, where we want to balance the risk of approving bad loans vs. rejecting good ones.

In [12]:
results = {}

print("🚀 Starting model training loop...")

for name, model in models.items():
    # Create a pipeline for the specific model
    # This ensures the preprocessor runs immediately before the model trains
    final_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        (name, model)
    ])
    
    # Train the model
    final_pipeline.fit(X_train, y_train)
    
    # Predict on Validation set
    y_pred = final_pipeline.predict(X_valid)

    # Store metrics
    results[name] = {
        "Accuracy": accuracy_score(y_valid, y_pred),
        "F1 Score": f1_score(y_valid, y_pred, average='weighted'),
        "Average Precision": average_precision_score(y_valid, y_pred) 
    }
    
    print(f"   ✅ {name} trained.")

print("🏁 Loop finished.")

🚀 Starting model training loop...
   ✅ Logistic Regression trained.
   ✅ Ridge Classifier trained.
   ✅ SGD Classifier trained.
   ✅ KNN trained.
   ✅ SVM trained.
   ✅ Decision Tree trained.
   ✅ Random Forest trained.
   ✅ Extra Trees trained.
   ✅ AdaBoost trained.
   ✅ Gradient Boosting trained.
   ✅ GaussianNB trained.
   ✅ BernoulliNB trained.
   ✅ Linear Discriminant Analysis trained.
   ✅ Quadratic Discriminant Analysis trained.
🏁 Loop finished.


c:\Users\yashd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
c:\Users\yashd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(


## 9. 🏆 Performance Leaderboard
We convert our results dictionary into a Pandas DataFrame to easily compare the models. We sort by **Accuracy** to see the top performers.

In [13]:
# Convert results to DataFrame and transpose so models are rows
results_df = pd.DataFrame(results).T

# Sort by Accuracy to find the best models
results_df = results_df.sort_values(by='Accuracy', ascending=False)

# Display the top 10 models
results_df.head(10)

,Accuracy,F1 Score,Average Precision
BernoulliNB,0.818182,0.796131,0.795990
Ridge Classifier,0.808081,0.782492,0.786745
Gradient Boosting,0.808081,0.790994,0.797369
Linear Discriminant Analysis,0.808081,0.782492,0.786745
SVM,0.797980,0.768464,0.777714
Random Forest,0.787879,0.772939,0.789336
AdaBoost,0.787879,0.759596,0.773835
Logistic Regression,0.767677,0.761606,0.792158
KNN,0.747475,0.699692,0.739964
Extra Trees,0.747475,0.737586,0.773458


## 10. 🎛️ Hyperparameter Tuning
Based on the screening results, we select the top candidates for fine-tuning. We use **GridSearchCV** to exhaustively search through a specified parameter grid to find the optimal configuration for each model.

**Selected Models for Tuning:**
1.  **Bernoulli Naive Bayes:** performed surprisingly well; we will tune the smoothing parameter (`alpha`).
2.  **Extra Trees:** A strong ensemble method; we will tune the number of trees and split criteria.
3.  **Decision Tree:** Included as a simpler baseline to compare against the ensembles.
4.  **Support Vector Machine (SVM):** A robust classifier; we will tune the regularization (`C`) and kernel type.

In [14]:
# Parameter grid for top-performing models
model_params = {
    
    # 1. Bernoulli Naive Bayes
    "BernoulliNB": {
        "model": BernoulliNB(),
        "params": {
            "model__alpha": [0.01, 0.1, 0.5, 1.0, 5.0, 10.0],
            "model__fit_prior": [True, False],
            "model__binarize": [0.0, 0.5, 1.0]  # added optional parameter
        }
    },

    # 2. Ridge Classifier
    "Ridge Classifier": {
        "model": RidgeClassifier(),
        "params": {
            "model__alpha": [0.1, 1.0, 10.0, 100.0],  # Regularization strength
            "model__solver": ["auto", "svd", "lsqr", "sag"]
        }
    },
    
    # 3. Support Vector Machine (SVM)
    "SVM": {
        "model": SVC(),
        "params": {
            "model__C": [0.1, 1, 10, 100],             # Regularization parameter
            "model__kernel": ["linear", "rbf"],        # Kernel type
            "model__gamma": ["scale", "auto"]          # Kernel coefficient
        }
    },

    # 4. Gradient Boosting
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(),
        "params": {
            "model__learning_rate": [0.01, 0.05, 0.1],
            "model__n_estimators": [100, 200, 300],
            "model__max_depth": [2, 3, 5],
            "model__min_samples_split": [2, 5],
            "model__min_samples_leaf": [1, 2]
        }
    },

    # 5. Linear Discriminant Analysis
    "Linear Discriminant Analysis": {
        "model": LinearDiscriminantAnalysis(),
        "params": {
            # If using 'svd' → shrinkage MUST be None
            "model__solver": ["svd"],
            "model__shrinkage": [None]
            
            # If using 'lsqr' or 'eigen' → shrinkage MUST be provided
            # "model__solver": ["lsqr", "eigen"],
            # "model__shrinkage": ["auto", 0.1, 0.5]
        }
    }
}

print("✅ Updated Parameter Grids for Best Models")


✅ Updated Parameter Grids for Best Models


## 11. 🔍 Running GridSearchCV
We loop through the selected models. For each combination of parameters, we use **5-Fold Cross-Validation**. This means the training data is split into 5 chunks; the model trains on 4 and tests on 1, rotating 5 times. This ensures the "Best Score" is reliable and not just a fluke.

In [15]:
results_tuning = {}

print("🚀 Starting Grid Search...")

for name, mp in model_params.items():
    # Construct the pipeline
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', mp["model"])
    ])
    
    # Initialize Grid Search
    # cv=5: 5-fold cross-validation
    # scoring='accuracy': optimizing for accuracy
    clf = GridSearchCV(pipe, mp["params"], cv=5, scoring='average_precision', error_score=np.nan)
    
    # Fit on the training set
    clf.fit(X_train, y_train)
    
    # Store the best results
    results_tuning[name] = {
        "Best Parameters": clf.best_params_,
        "Best Accuracy": clf.best_score_
    }
    
    print(f"   ✅ {name} tuned. Best Score: {clf.best_score_:.4f}")

print("🏁 Tuning finished.")

🚀 Starting Grid Search...
   ✅ BernoulliNB tuned. Best Score: 0.8375
   ✅ Ridge Classifier tuned. Best Score: 0.8393
   ✅ SVM tuned. Best Score: 0.8247
   ✅ Gradient Boosting tuned. Best Score: 0.8055
   ✅ Linear Discriminant Analysis tuned. Best Score: 0.8157
🏁 Tuning finished.


## 12. 📊 Tuning Results & Model Selection
Let's view the best parameters found for each model.

In [16]:
# Convert to DataFrame for easy viewing
results_tuning_df = pd.DataFrame(results_tuning).T
results_tuning_df

,Best Parameters,Best Accuracy
BernoulliNB,"{'model__alpha': 10.0, 'model__binarize': 0.0,...",0.837537
Ridge Classifier,"{'model__alpha': 100.0, 'model__solver': 'auto'}",0.839289
SVM,"{'model__C': 1, 'model__gamma': 'auto', 'model...",0.824702
Gradient Boosting,"{'model__learning_rate': 0.05, 'model__max_dep...",0.805467
Linear Discriminant Analysis,"{'model__shrinkage': None, 'model__solver': 's...",0.815706


## 13. 🔁 Final Evaluation – Best Models After Hyperparameter Tuning

Using the best parameters from GridSearchCV, we trained and evaluated all models again on the test set.

For each model, we computed:
- Accuracy  
- Precision  
- Recall  
- F1 Score  
- Confusion Matrix  
- Classification Report  

A performance comparison table was generated to select the final model for deployment.


In [17]:
# Best parameters from GridSearchCV
best_models = {
    "BernoulliNB": BernoulliNB(alpha=10.0, binarize=0.0, fit_prior=True),

    "Ridge Classifier": RidgeClassifier(alpha=100.0, solver="auto"),

    "SVM": SVC(C=1, gamma="auto", kernel="rbf", probability=True),   # probability=True → For ROC Curve

    "Gradient Boosting": GradientBoostingClassifier(learning_rate=0.05,
                                                    max_depth=2,
                                                    min_samples_leaf=1,
                                                    min_samples_split=5,
                                                    n_estimators=100,
                                                    random_state=42
                                                    ),

    "Linear Discriminant Analysis": LinearDiscriminantAnalysis(shrinkage=None,
                                                               solver="svd"
                                                    ),

    "Random Forest": RandomForestClassifier(n_estimators=100, 
                                            max_depth=None,
                                            min_samples_leaf=2,
                                            min_samples_split=5,
                                            class_weight='balanced',
                                            random_state=42
                                                    )
}

#-------------------------------------------------------------

results_test = {}
print("🚀 Evaluating ALL Best Models on Test Set...\n")

for name, model in best_models.items():
    # Create pipeline
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Train model
    pipe.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = pipe.predict(X_test)

    # Store results
    results_test[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred, average='weighted'),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred)
    }

    # Print classification report & confusion matrix
    print(f"📌 {name} - Classification Report:")
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("-" * 60)

print("\n🏁 Evaluation Completed!")


🚀 Evaluating ALL Best Models on Test Set...

📌 BernoulliNB - Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.55      0.70        38
           1       0.83      0.99      0.90        85

    accuracy                           0.85       123
   macro avg       0.89      0.77      0.80       123
weighted avg       0.87      0.85      0.84       123


Confusion Matrix:
[[21 17]
 [ 1 84]]
------------------------------------------------------------
📌 Ridge Classifier - Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.55      0.70        38
           1       0.83      0.99      0.90        85

    accuracy                           0.85       123
   macro avg       0.89      0.77      0.80       123
weighted avg       0.87      0.85      0.84       123


Confusion Matrix:
[[21 17]
 [ 1 84]]
------------------------------------------------------------
📌 SVM - Classificati

In [18]:
pd.DataFrame(results_test).T.sort_values(by="Accuracy", ascending=False)

,Accuracy,F1 Score,Precision,Recall
BernoulliNB,0.853659,0.840441,0.831683,0.988235
Ridge Classifier,0.853659,0.840441,0.831683,0.988235
SVM,0.853659,0.840441,0.831683,0.988235
Linear Discriminant Analysis,0.853659,0.840441,0.831683,0.988235
Gradient Boosting,0.845528,0.835078,0.836735,0.964706
Random Forest,0.796748,0.792495,0.833333,0.882353


## 14. 🏁 Final Model Selection

After evaluating the tuned models on the **test dataset**, four models demonstrated strong and consistent performance:
- Support Vector Machine (SVM)
- Bernoulli Naive Bayes (BernoulliNB)
- Ridge Classifier
- Linear Discriminant Analysis (LDA)

These models were shortlisted because:
✔ They achieved high F1-scores  
✔ They produced **only 1 False Negative** (critical for loan approval)  
✔ They showed excellent **recall for the approved class (Class 1)**  
✔ Suitable for business use — minimal rejection of genuine loan applicants  

---

### 🔍 Business-Critical Evaluation Criteria

| Critical Metric | Why It Matters in Loan Approval | Top Models |
|-----------------|----------------------------------|-------------|
| **False Negatives (FN)** | Rejecting valid applicants leads to business loss | SVM, BernoulliNB, Ridge, LDA |
| **Recall (Class 1)** | Must approve almost all eligible applicants | SVM (Recall = 0.99) |
| **F1-Score** | Balance between approval rate & risk | SVM & BernoulliNB |
| **Interpretability** | Financial decisions require transparency | Ridge & BernoulliNB |
| **Deployment Speed** | Needs real-time API usage | BernoulliNB |

From a **domain (banking) perspective**, minimizing **False Negatives** is MOST important:
> 🚨 Rejecting an eligible customer is worse than approving a slightly risky customer.

---

### 🏆 Final Decision — Selected Model

We selected **Support Vector Machine (SVM)** as the **final model** for deployment because:
- It achieved **only 1 False Negative**
- Very high **recall (0.99)** for approved loans
- Strong balance between precision and recall
- Handles high-dimensional one-hot encoded data effectively

---

### ⚙ Tuned Hyperparameters (From GridSearchCV)

```python
model = SVC(
    C=1,
    kernel='rbf',
    gamma='auto',
    probability=True
)

### **Next Steps:**
In the final evaluation notebook (`3. Model_Training_Test.ipynb`), we will:
1.  Instantiate the SVM with these specific parameters.
2.  Train it on the full training data.
3.  Evaluate it on the unseen **Test Set** (X_test) to get the final performance metrics.